<a href="https://colab.research.google.com/github/cleysonl/Tensorflow2.0/blob/master/Transfer_Learning_and_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transfer Learning and Fine tuning with Tensorflow2.0**

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 47kB/s 
     |████████████████████████████████| 419kB 43.6MB/s 
     |████████████████████████████████| 3.0MB 59.4MB/s 


In [2]:
!pip install tqdm

### **Downloading the Dogs vs. Cats Dataset**

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2019-11-09 09:12:52--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 2a00:1450:4013:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  43.3MB/s    in 1.5s    

2019-11-09 09:12:54 (43.3 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



### **Import project dependencies**

In [4]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

'2.0.0-alpha0'

### **Unzipping the Dogs vs Cats Dataset**

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(file=dataset_path, mode='r')
zip_object.extractall('./')
zip_object.close()

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## **Building the model**

In [0]:
IMG_SHAPE = (128, 128, 3)

In [8]:
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE, include_top = False, weights="imagenet")

9412608/9406464 [==============================] - 1s 0us/step


In [9]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### **Freezing the base model**

In [0]:
base_model.trainable = False 

### **Defining the custom head for our network**

In [11]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [13]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation ='sigmoid')(global_average_layer)

### **Defining the model**

In [0]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

### **Compiling the model**

In [0]:
optimizer = tf.keras.optimizers.RMSprop(lr=0.0001)
loss = 'binary_crossentropy'
metrics = ['accuracy']

model.compile(optimizer=optimizer, loss= loss, metrics = metrics)

### **Creating Data Generators**

In [0]:
data_gen_train = ImageDataGenerator(rescale= 1/255.)
data_gen_valid = ImageDataGenerator(rescale= 1/255.)

In [19]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=120, class_mode='binary')

Found 2000 images belonging to 2 classes.


In [20]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=120, class_mode='binary')

Found 1000 images belonging to 2 classes.


### **Training the model**

In [21]:
model.fit_generator(train_generator, epochs = 5, validation_data=valid_generator)

Epoch 1/5
17/17 [==============================] - 40s 2s/step - loss: 0.2340 - accuracy: 0.9035 - val_loss: 0.0870 - val_accuracy: 0.9660
Epoch 2/5
17/17 [==============================] - 7s 431ms/step - loss: 0.0239 - accuracy: 0.9975 - val_loss: 0.0847 - val_accuracy: 0.9660
Epoch 3/5
17/17 [==============================] - 7s 419ms/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 0.0804 - val_accuracy: 0.9680
Epoch 4/5
17/17 [==============================] - 7s 421ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0966 - val_accuracy: 0.9660
Epoch 5/5
17/17 [==============================] - 7s 422ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.1112 - val_accuracy: 0.9680


### **Transfer learning model evaluation**

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [23]:
print('Accuracy after transfer learning: {}'.format(valid_accuracy))

Accuracy after transfer learning: 0.9679999947547913


## **Fine Tunning**

### **Un-freeze a few top layers from the model**

In [0]:
base_model.trainable = True

In [26]:
print('Number od layers in the base model: {}'.format(len(base_model.layers)))

Number od layers in the base model: 155


In [0]:
fine_tune_at = 100

In [0]:
# Freezes weights from layer 1 to 149
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

### **Compiling the model for fine-tuning**

In [0]:
model.compile(optimizer= tf.keras.optimizers.RMSprop(lr=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

### **Fine Tuning**

In [31]:
model.fit_generator(train_generator,
                    epochs = 5,
                    validation_data=valid_generator)

Epoch 1/5
17/17 [==============================] - 23s 1s/step - loss: 5.3464e-04 - accuracy: 1.0000 - val_loss: 0.2360 - val_accuracy: 0.9530
Epoch 2/5
17/17 [==============================] - 7s 406ms/step - loss: 1.5745e-04 - accuracy: 1.0000 - val_loss: 0.2345 - val_accuracy: 0.9530
Epoch 3/5
17/17 [==============================] - 7s 402ms/step - loss: 8.3439e-05 - accuracy: 1.0000 - val_loss: 0.1515 - val_accuracy: 0.9710
Epoch 4/5
17/17 [==============================] - 7s 395ms/step - loss: 4.5757e-05 - accuracy: 1.0000 - val_loss: 0.1116 - val_accuracy: 0.9680
Epoch 5/5
17/17 [==============================] - 7s 401ms/step - loss: 4.5951e-05 - accuracy: 1.0000 - val_loss: 0.1836 - val_accuracy: 0.9650


### **Evaluating the fine tuned model**

In [32]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

9/9 [==============================] - 3s 298ms/step - loss: 0.1836 - accuracy: 0.9650


In [33]:
print('Accuracy after transfer learning: {}'.format(valid_accuracy))

Accuracy after transfer learning: 0.9649999737739563
